In [47]:
import time
import pandas as pd
from datetime import date
from langchain_core.messages import SystemMessage

from llm_political_analysis.modules.prompts import personas, encouragements

In [48]:
ches_results = pd.read_csv('../data/ches_scores/CHES cal-test-data.csv')
ches_results["merge_filename"] = ches_results["file"].apply(lambda x:f"{x}_en_translation.txt")
ches_results

,year,country,party,file,EU,TaxSpend,Social_lifestyle,Immigration,Decentralization,Environment,Cal_Test,merge_filename
0,2019,AUS,SPO,Austria - AU 2019 - AU 2019 SDP,6.1,2.5,2.4,4.7,6.0,4.9,C,Austria - AU 2019 - AU 2019 SDP_en_translation...
1,2014,BEL,N-VA,Belgium - BEL 2014 - BEL 2014 New Flemish All...,5.0,9.0,3.6,7.6,2.0,7.6,C,Belgium - BEL 2014 - BEL 2014 New Flemish All...
2,2010,CZE,ODS,Czech - CZ 2010 - CZ 2010 Civ Dem ODS,3.0,8.2,5.4,6.5,6.3,8.1,C,Czech - CZ 2010 - CZ 2010 Civ Dem ODS_en_trans...
3,2019,DK,SF,Denmark - DEN 2019 SPP SF,5.1,1.8,1.6,4.3,5.1,3.4,C,Denmark - DEN 2019 SPP SF_en_translation.txt
4,2014,HUN,JOBBIK,Hungary - HUN 2014 - HUN 2010 Mvmnt for better...,1.2,4.3,9.8,9.3,5.1,5.8,C,Hungary - HUN 2014 - HUN 2010 Mvmnt for better...
5,2010,NETH,GL,Netherlands - NL 2010 - NL 2010 Green GL,6.2,2.4,1.4,2.6,4.5,1.5,C,Netherlands - NL 2010 - NL 2010 Green GL_en_tr...
6,2019,POL,PO,Poland - POL 2019 Civic Coalition PO,6.7,5.3,4.2,4.6,2.8,5.1,C,Poland - POL 2019 Civic Coalition PO_en_transl...
7,2010,SWE,M,Sweden - SWE 2010 - SWE 2010 Moderate,6.3,7.6,3.2,5.4,5.2,5.9,C,Sweden - SWE 2010 - SWE 2010 Moderate_en_trans...
8,2006,AUS,Freedom Party of Austria,Austria - AU 2006 - AU 2006 FPO,1.8,5.7,9.5,9.8,6.2,NaN,T,Austria - AU 2006 - AU 2006 FPO_en_translation...
9,2010,BEL,ECOLO,Belgium - BEL 2010 - BEL 2010 Green Ecolo,6.0,1.5,1.2,1.0,5.8,1.1,T,Belgium - BEL 2010 - BEL 2010 Green Ecolo_en_t...


In [49]:
def parse_persona_encouragement(msg: SystemMessage):
    [system_msg, _] = msg
    persona = None
    encouragement = None
    for i, per in enumerate(personas):
        if system_msg.content.find(per) >= 0:
            if persona is not None:
                raise ValueError("Persona already exist.")
            persona = i
    for i, enc in enumerate(encouragements):
        if system_msg.content.find(enc) >= 0:
            if encouragement is not None:
                raise ValueError("Encouragement already exist.")
            encouragement = i
    if persona is None or encouragement is None:
        raise ValueError(f'No persona or encouragement found.')
    return persona, encouragement


def add_prompt_cls(row):
    persona, encouragement = parse_persona_encouragement(eval(row["prompt"]))
    return persona+1, encouragement+1, f"{persona+1}-{encouragement+1}"

In [50]:
path = [
    r"C:\Users\Jinshuai\Workspace\data\llm-expert\output\2024-08-09-1723164639-gemini_analysis_on_translation\analyze_results.xlsx",
    r"C:\Users\Jinshuai\Workspace\data\llm-expert\output\2024-08-09-1723164677-gpt_analysis_on_translation\analyze_results.xlsx",
    r"C:\Users\Jinshuai\Workspace\data\llm-expert\output\2024-08-09-1723164830-claude_analysis_on_translation\analyze_results.xlsx"
]

In [51]:
df_result = pd.concat([pd.read_excel(f) for f in path])
df_result["merge_filename"] = df_result.file.apply(lambda x: x.replace(r'/home/jinshuai/project/llm-political-analysis/llm_political_analysis/data/output/2024-08-09-1723163805-gpt_summary_translation/summary_standard_multi_issue__', ''))
df_result[["persona", "encouragement", "prompt_cls"]] = df_result.apply(add_prompt_cls, result_type="expand", axis=1)
df_result

,file,issue,model,score,error_message,prompt,created_at,merge_filename,persona,encouragement,prompt_cls
0,/home/jinshuai/project/llm-political-analysis/...,european_union,gemini-1.5-pro-001,1.0,NaN,"[SystemMessage(content=""You are an expert soci...",2024-08-09 00:50:45.731,Slovakia - SLO 2006 - SLO 2006 Direction_en_tr...,1,1,1-1
1,/home/jinshuai/project/llm-political-analysis/...,european_union,gemini-1.5-pro-001,1.0,NaN,"[SystemMessage(content=""You are an expert soci...",2024-08-09 00:50:45.731,Slovakia - SLO 2006 - SLO 2006 Direction_en_tr...,1,2,1-2
2,/home/jinshuai/project/llm-political-analysis/...,european_union,gemini-1.5-pro-001,1.0,NaN,"[SystemMessage(content=""You are an expert soci...",2024-08-09 00:50:45.731,Slovakia - SLO 2006 - SLO 2006 Direction_en_tr...,1,3,1-3
3,/home/jinshuai/project/llm-political-analysis/...,european_union,gemini-1.5-pro-001,1.0,NaN,"[SystemMessage(content=""You are highly intelli...",2024-08-09 00:50:45.731,Slovakia - SLO 2006 - SLO 2006 Direction_en_tr...,2,1,2-1
4,/home/jinshuai/project/llm-political-analysis/...,european_union,gemini-1.5-pro-001,1.0,NaN,"[SystemMessage(content=""You are highly intelli...",2024-08-09 00:50:45.731,Slovakia - SLO 2006 - SLO 2006 Direction_en_tr...,2,2,2-2
...,...,...,...,...,...,...,...,...,...,...,...
805,/home/jinshuai/project/llm-political-analysis/...,decentralization,claude-3-5-sonnet-20240620,3.0,NaN,[SystemMessage(content='You are highly intelli...,2024-08-09 01:08:32.624,Finland - FIN 2019 - FIN 2019 Nat Coal KOK_en_...,2,2,2-2
806,/home/jinshuai/project/llm-political-analysis/...,decentralization,claude-3-5-sonnet-20240620,3.0,NaN,[SystemMessage(content='You are highly intelli...,2024-08-09 01:08:32.624,Finland - FIN 2019 - FIN 2019 Nat Coal KOK_en_...,2,3,2-3
807,/home/jinshuai/project/llm-political-analysis/...,decentralization,claude-3-5-sonnet-20240620,3.0,NaN,[SystemMessage(content='You are a professor of...,2024-08-09 01:08:32.624,Finland - FIN 2019 - FIN 2019 Nat Coal KOK_en_...,3,1,3-1
808,/home/jinshuai/project/llm-political-analysis/...,decentralization,claude-3-5-sonnet-20240620,3.0,NaN,[SystemMessage(content='You are a professor of...,2024-08-09 01:08:32.624,Finland - FIN 2019 - FIN 2019 Nat Coal KOK_en_...,3,2,3-2


In [52]:
df_res = df_result.merge(ches_results, on="merge_filename")[[
    "file_y", "issue", "model", "score", "error_message", "prompt", "created_at", "persona", "encouragement", "prompt_cls", "year", "country", "party", "EU", "TaxSpend", "Social_lifestyle", "Immigration", "Decentralization", "Environment", "Cal_Test"
]]

In [53]:
df_res.to_csv(f"../data/results/result_on_translation_summary-{date.today()}-{int(time.time())}.csv", index=False)